In [1]:
from nnsight import LanguageModel

/share/u/caden/.conda/envs/interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = LanguageModel("gpt2", device_map="auto")

In [4]:
with model.trace("Hello") as tracer:
    pass